# Simple card game IA

We will describe in this notebook a very simple 2 players card game and a way to implement an IA to play this game using an **artificial neural network** (we will use a multi layer perceptron model)

The IA will be trained only by playing, which means that no "expert" human player will implement strategies.

The only information about the game given to the IA is a function which indicates to the IA the cards allowed to play.

## The game
The game is a very basic 2 player "Trick-taking" card game :

### General description
* There are 2 colors (color 0 and color 1)
* Each color has N cards (valued from 0 to N-1)
* The game is composed of N "tricks"
* Player 0 starts to play for the first trick
* The winner of a trick starts to play the next trick

### Trick rules
* The first player can play any card
* If the second player has cards with the same color, ** he has to play the same color**
* Otherwise he plays the other color

### Trick winner
* If the second player plays the same color. **The card with the highest value wins the trick**
* If the second player does not have the same color. **The first player wins**

### Example
* N = 3
* We denote (1,0) card 1 of color 0
* Player 0's hand = [(0, 0), (2, 0), (1, 1)]
* Player 1's hand = [(1, 0), (0, 1), (2, 1)]
 * Player 0 plays (2,0)
 * Player 1 plays (1,0)
   * Player 0 wins the trick since 2 > 1 and start to play next trick
 * Player 0 plays (0,0)
 * Player 1 plays (0,1)
   * Player 0 wins the trick since player 1 with no color 0, played color 1
 * Player 0 plays (1, 1)
 * Player 1 plays (2, 1)
   * Player 1 wins the trick since 2 > 1
* At the end of the game, player 0 wins 2 tricks, player 1 wins 1 trick

## Strategies comparison

In order to compare strategies, the players will play a twice :
* Player 0 with hand 0, Player 1 with hand 1, Player 0 stats
* Player 1 with hand 0, Player 0 with hand 1, Player 1 stats

Then we compare both games :
* If player 0 wins more tricks with hand 0 than player 1 with hand 0, player 0 wins the whole game
* If player 1 wins more tricks with hand 0 than player 0 with hand 0, player 1 wins the whole game
* Otherwise its draw

## IA implementation
We will implement a simple MLP model using keras library

In [10]:
# Importation
import numpy as np

import keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD, RMSprop

In [5]:
# General function to generate hands
def generateDeck(N=10, P=2):
    deck = np.zeros((P*N, 2))
    for i in range(0,P):
        deck[i * N:i * N + N,0] = np.arange(N)
        deck[i * N:i * N + N,1] = i
    return deck

def gererateHand(N=10, P=2, seed=0):
    """ Generate N cards hands for P players.
    The function returns the deck indices"""
    np.random.seed(seed)
    deck = np.arange(N*P)
    np.random.shuffle(deck)
    return np.reshape(deck, (P,-1))

## Rules
Here is the rule function that indicates to the player the cards they can play, according to their hand and the previous trick.

In [6]:
def playable(hand, deck, trick=[]):
    """Returns indices of playable cards """
    allrange = np.arange(len(hand))
    if len(trick) == 0:
        return allrange # First to play

    color = deck[trick[0]][1]
    colors = allrange[deck[hand][:,1] == color]
    if len(colors) > 0: # Forced to play same color
        return colors
    else: # Does not have the color
        return allrange

## Random Strategy

Here is the **random strategy**.

The player plays the first card he founds and since cards are distributed in a random way (see gererateHand()) its equivalent to a random selection.


In [7]:
def firstPlay(hand, deck, trick=[]):
    choices = playable(hand, deck, trick)
    return choices[0]

## First implementation

We will first implement 2 players playing the same random strategy.
Since everything is seeded, all the game should end with a **draw** result

In [12]:
def playTrick(hands, hist, deck, dealer=0):
    """Let the players play one trick """
    tricks = []
    P = len(hands)
    for player in range(dealer, dealer+P):
        player %= P
        if player == 0:
            card_index = firstPlay(hands[player], deck, tricks) # Player 1 plays "firstPlay strategy"
        elif player == 1:
            card_index = firstPlay(hands[player], deck, tricks) # Player 2 plays "firstPlay strategy"
        tricks.append(hands[player][card_index])
    return tricks


def getWinner(trick, deck):
    """Computes which player wins the trick """
    good_trump = deck[trick][:,1] == deck[trick[0],1] # players with the right color
    winner = 0
    for i in range(1, len(trick)):
        if good_trump[i] and deck[trick[i]][0] > deck[trick[winner]][0]:
            winner = i
    return winner


def getHands(hands, played_cards):
    """ Extract current hands from played cards"""
    cleanHand = []
    for hand in hands:
        cleanHand.append(hand[np.array([card not in played_cards for card in hand])])
    return cleanHand


def play(hands, deck, dealer=0):
    """Plays N tricks and returns played_card order, first player, and winner"""
    played_cards = []
    who_play = []
    winner = []
    
    for i in range(0, len(hands[0])): # N tricks
        # P players starting from dealer
        who_play += list((np.arange(0, len(hands)) + dealer) % len(hands))

        trick = playTrick(getHands(hands, played_cards), played_cards, deck, dealer=dealer)      
        played_cards += trick # Save played card history
        
        winner_rel = getWinner(trick, deck)
        dealer = (dealer + winner_rel) % len(hands) # Compute which player will start next
        winner.append(dealer)
    return (played_cards, winner, who_play)


def shiftHand(hands):
    """Exchange hands so that each player can play each hand"""
    hands2 = np.zeros(hands.shape, dtype=hands.dtype )
    hands2[0] = hands[-1]
    hands2[1:] = hands[0:-1]
    return hands2

## Lets play
Now both players can play the first random strategy
Lets generate a deck, play several games and compute the winner.
According to the implementation, all games must be draw

In [14]:
deck0 = generateDeck(N=10, P=2)
num_of_games = 1000
w1 = 0
w2 = 0
d = 0

for i in range(0, num_of_games):
    # Lets play num_of_games games
    hand0 = gererateHand(N=10, P=2, seed=i) # seed is fixed

    (hist, win, who_play) = play(hand0, deck0, 0)
    (shift_hist, shift_win, shift_who_play) = play(shiftHand(hand0), deck0, 1)

    if sum([w == 0 for w in win]) > sum([w == 1 for w in shift_win]):
        w1 += 1
    elif sum([w == 0 for w in win]) < sum([w == 1 for w in shift_win]):
        w2 += 1
    else:
        d += 1
print(w1 / num_of_games * 100, w2 / num_of_games * 100, d / num_of_games * 100)


0.0 0.0 100.0


Expected result:

0.0 0.0 100.0

=> 100% of played games ends with draw (as expected)